In [68]:
import os
import glob
import pandas as pd
from collections import Counter

In [73]:
def count_to_tpm(count_df, gene_lengths):
    rpk = count_df.div(gene_lengths, axis=0)
    scaling_factor = rpk.sum(axis=0)
    tpm = rpk.div(scaling_factor, axis=1) * 1e6
    return tpm

def tpm_to_count(tpm_df, gene_lengths):
    common_genes = tpm_df.index.intersection(gene_lengths.index)
    tpm_df = tpm_df.loc[common_genes]
    gene_lengths = gene_lengths.loc[common_genes]
    rpk = tpm_df.multiply(gene_lengths, axis=0) / 1000
    scaling_factors = rpk.sum(axis=0) / 1e6
    count_df = rpk.divide(scaling_factors, axis=1)
    count_df = count_df.round().astype(int)
    return count_df

In [70]:
xls_files = glob.glob("GS*.xlsx")
tsv_files = glob.glob("GS*.tsv")
csv_files = glob.glob("GS*.csv")
all_files = xls_files + tsv_files + csv_files
all_files

['GSE139889.xlsx',
 'GSE119629.tsv',
 'GSE150642.tsv',
 'GSM8291018.csv',
 'GSM8291019.csv',
 'GSM8291020.csv',
 'GSM8291021.csv',
 'GSM8291022.csv']

In [71]:
gene_lengths = pd.read_csv("gene_lengths.csv", index_col=0).squeeze()

In [92]:
dataframes = []

for idx, file in enumerate(all_files):
    print(f"Processing file {idx}: {file}")

    if file.endswith(".xlsx"):
        df = pd.read_excel(file)
    elif file.endswith(".csv"):
        df = pd.read_csv(file)
    elif file.endswith(".tsv"):
        df = pd.read_csv(file, sep="\t")
    else:
        continue
        
    sample_cols = [col for col in df.columns if col.startswith("S")]
    df = df[["GeneID"] + sample_cols].drop_duplicates(subset="GeneID")
    df.set_index("GeneID", inplace=True)

    if idx < 2:
        try:
            df = tpm_to_count(df, gene_lengths)
        except Exception as e:
            print(f"Error converting to TPM in file {file}: {e}")
            continue
    df.columns = [f"{col}_file{idx}" for col in df.columns]
    dataframes.append(df)
    
dataframes

Processing file 0: GSE139889.xlsx
Processing file 1: GSE119629.tsv
Processing file 2: GSE150642.tsv
Processing file 3: GSM8291018.csv
Processing file 4: GSM8291019.csv
Processing file 5: GSM8291020.csv
Processing file 6: GSM8291021.csv
Processing file 7: GSM8291022.csv


[             S1_file0  S2_file0  S3_file0  S4_file0  S5_file0  S6_file0  \
 GeneID                                                                    
 FBgn0000003         1         2         7         5         1         3   
 FBgn0000008       114       167       308       185        87         7   
 FBgn0000014         0         0         0         0         0         0   
 FBgn0000015         0         0         0         0         0         0   
 FBgn0000017        71        84       176        12         0        87   
 ...               ...       ...       ...       ...       ...       ...   
 FBgn0287695         0         0         0         0         0        18   
 FBgn0287696         4         2         9         0        15         0   
 FBgn0287720         0         0         0         0         0         0   
 FBgn0287724         0         0         0         0         0         0   
 FBgn0287725         0        12        30         3         0        10   
 
          

In [93]:
merged_df = dataframes[0]
for df in dataframes[1:]:
    merged_df = pd.merge(merged_df, df, on='GeneID', how='inner')
merged_df  

,S1_file0,S2_file0,S3_file0,S4_file0,S5_file0,S6_file0,S7_file0,S8_file0,S9_file0,S10_file0,...,S13_file1,S14_file1,S1_file2,S2_file2,S3_file2,S_file3,S_file4,S_file5,S_file6,S_file7
GeneID,,,,,,,,,,,,,,,,,,,,,
FBgn0000008,114,167,308,185,87,7,28,62,113,33,...,180,61,2811,1618,2026,7476,5336,6201,6655,4347
FBgn0000014,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,79,2,1,0
FBgn0000015,0,0,0,0,0,0,0,0,0,0,...,0,26,6,10,2,3,75,5,2,0
FBgn0000017,71,84,176,12,0,87,46,51,79,110,...,0,0,2276,1198,1419,10015,7798,9876,8840,7249
FBgn0000018,0,1,1,0,0,0,2,1,0,1,...,0,0,164,146,198,100,90,69,92,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FBgn0267728,2,0,0,0,0,0,0,4,0,5,...,0,0,149,60,60,153,101,97,155,106
FBgn0267790,94,111,19,5,0,16,0,84,0,9,...,14,380,2977,2290,2044,2121,1674,1538,1937,1522
FBgn0267791,45,39,32,38,0,156,0,99,10,12,...,0,6,4631,3629,3114,5452,4333,5375,5182,4186


In [94]:
merged_df.index.name = None
merged_df = merged_df.dropna()
merged_df

,S1_file0,S2_file0,S3_file0,S4_file0,S5_file0,S6_file0,S7_file0,S8_file0,S9_file0,S10_file0,...,S13_file1,S14_file1,S1_file2,S2_file2,S3_file2,S_file3,S_file4,S_file5,S_file6,S_file7
FBgn0000008,114,167,308,185,87,7,28,62,113,33,...,180,61,2811,1618,2026,7476,5336,6201,6655,4347
FBgn0000014,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,79,2,1,0
FBgn0000015,0,0,0,0,0,0,0,0,0,0,...,0,26,6,10,2,3,75,5,2,0
FBgn0000017,71,84,176,12,0,87,46,51,79,110,...,0,0,2276,1198,1419,10015,7798,9876,8840,7249
FBgn0000018,0,1,1,0,0,0,2,1,0,1,...,0,0,164,146,198,100,90,69,92,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FBgn0267728,2,0,0,0,0,0,0,4,0,5,...,0,0,149,60,60,153,101,97,155,106
FBgn0267790,94,111,19,5,0,16,0,84,0,9,...,14,380,2977,2290,2044,2121,1674,1538,1937,1522
FBgn0267791,45,39,32,38,0,156,0,99,10,12,...,0,6,4631,3629,3114,5452,4333,5375,5182,4186
FBgn0267792,0,0,0,0,0,0,0,3,0,0,...,0,0,1097,919,781,363,371,318,315,294


In [95]:
merged_df.to_pickle("express_data.pkl")